## HyDE, streaming, error handling, Langsmith

In [ ]:
# Setup & Imports
import time

from src.langchain.chains import MovieRAGChain
from src.langchain.observability import setup_langsmith
from src.langchain.prompts import ZERO_SHOT_QA_PROMPT
from src.langchain.streaming import CitationStreamingWrapper

print("✓ Imports successful")

In [ ]:
# Configuration
PLOTS_PATH = "/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv"
REVIEWS_PATH = "/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv"
MAX_MOVIES = 500  # Limit for faster demos

In [3]:
def run(chain, query, strategy_name):
    start = time.time()
    result = chain.query(query)
    time_t = time.time() - start

    print(f"\nStrategy: {strategy_name}")
    print(f"\nQuery: {query}")
    print(f"\nTime: {time_t:.2f}s")
    print(f"\nAnswer: {result['answer']}")
    print(f"\nTop 5 Retrieved:")
    if 'hyde_query' in result['sources'][0]['metadata']:
            print(f"\n  hyde query: {result['sources'][0]['metadata']['hyde_query']}")
    for src in result['sources'][:5]:
        score = src['metadata'].get('score', 'N/A')
        print(f"\n  {src['metadata']['movie_title']} - {src['metadata']['release_year']}(initial score={score})")
        print(f"\n content: {src['content']}...")
        if 'rerank_score' in src['metadata']:
            print(f"\n  rerank score: {src['metadata']['rerank_score']:.3f}")
    if 'citations' in result:
        print(f"\n  Citations: {', '.join(result.get('citations', [])[:3])}")

#### Baseline chain

In [12]:
print("="*60)
print("DEMO 1: Baseline RAG")
print("="*60)

# Build baseline chain
baseline_chain = MovieRAGChain(
    plots_path=PLOTS_PATH,
    reviews_path=REVIEWS_PATH,
    max_movies=MAX_MOVIES,
    use_custom_retriever=True,
    use_custom_chunk=True,
    custom_prompt=ZERO_SHOT_QA_PROMPT,
    k=5,
)

baseline_chain.build()
run(baseline_chain, "What movies are similar to Inception?", "Baseline RAG")

DEMO 1: Baseline RAG
✓ MovieRAGChain initialized
  Retriever type: custom
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...


2025-10-14 14:48:09,014 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: mps
2025-10-14 14:48:09,015 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2. Chunking with custom func...

Chunking documents...
Chunked 883 docs → 10114 chunks using 'sentence' strategy.

Building base retriever...

3. Building custom retriever...
Loading embedding model: text-embedding-3-small (provider: openai)
✓ Model loaded (dimension: 1536)
✓ FaissDenseRetriever initialized (index_type=flat)
Generating embeddings for 10114 documents...


2025-10-14 14:48:14,873 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:48:19,508 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:48:24,214 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:48:29,058 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:48:34,150 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:48:37,217 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-14 14:48:37,221 - openai._base_client - INFO - Retrying request to /embeddings in 8.479000 seconds
2025-10-14 14:48:49,451 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:48:53,310 - httpx - INFO - HTTP Request: POST https://api.o

Embeddings generated
Saving index...
✓ Added 10114 documents to FAISS index
  Index size: 10114

5. Creating QA chain...

✓ RAG Pipeline Built!


2025-10-14 14:49:02,511 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:49:05,276 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Strategy: Baseline RAG

Query: What movies are similar to Inception?

Time: 3.28s

Answer: Based on the provided information, "Transcendence" is mentioned as having the audacity of Christopher Nolan's "Inception," although it lacks the same credibility and depth. While "Transcendence" is not as well-received, it shares themes of technology and consciousness that may appeal to fans of "Inception." Other films that explore similar concepts of reality, technology, and the human mind include "The Lawnmower Man" and "Her," although "Transcendence" is criticized for being a lesser version of such films.

Top 5 Retrieved:

  Transcendence - 2014(initial score=-1.0555334091186523)

 content: Review: In his first film as director, acclaimed cinematographer Wally Pfister (The Dark Knight Rises, Inception) has made a movie that predictably lo...

  Transcendence - 2014(initial score=-1.0730133056640625)

 content: Review: has all the audacity of Christopher Nolan's Inception, without the credibi

#### Rag + Reranking

In [13]:
print("="*60)
print("DEMO 2: RAG + Reranking")
print("="*60)

# Build chain with reranking
reranking_chain = MovieRAGChain(
    plots_path=PLOTS_PATH,
    reviews_path=REVIEWS_PATH,
    max_movies=MAX_MOVIES,
    use_custom_retriever=True,
    use_custom_chunk=True,
    custom_prompt=ZERO_SHOT_QA_PROMPT,
    k=5,
    use_reranking=True,
    reranker_cfg={'type':'llm'},
    initial_k=20,    
)

reranking_chain.build()

# Test reranking
run(reranking_chain, "Find dark psychological thrillers", "RAG + Reranking")

DEMO 2: RAG + Reranking
✓ MovieRAGChain initialized
  Retriever type: custom + reranking
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...


2025-10-14 14:50:10,389 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: mps
2025-10-14 14:50:10,390 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2. Chunking with custom func...

Chunking documents...
Chunked 883 docs → 10114 chunks using 'sentence' strategy.

Building base retriever...

3. Building custom retriever...
Loading embedding model: text-embedding-3-small (provider: openai)
✓ Model loaded (dimension: 1536)
✓ FaissDenseRetriever initialized (index_type=flat)
Generating embeddings for 10114 documents...


2025-10-14 14:50:18,133 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:50:22,896 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:50:28,288 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:50:33,135 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:50:36,923 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:50:38,766 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-14 14:50:38,770 - openai._base_client - INFO - Retrying request to /embeddings in 8.812000 seconds
2025-10-14 14:50:51,364 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:50:53,292 - httpx - INFO - HTTP Request: POST https://api.o

Embeddings generated
Saving index...
✓ Added 10114 documents to FAISS index
  Index size: 10114

4. Adding reranking: 20 → 5 docs...
✓ LLMReranker initialized: gpt-4o-mini
✓ Created LLM reranking retriever (top_k=5)

5. Creating QA chain...

✓ RAG Pipeline Built!


2025-10-14 14:51:04,165 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 14:51:07,543 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 14:51:07,950 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 14:51:08,382 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 14:51:08,873 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 14:51:12,045 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 14:51:12,457 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 14:51:13,070 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 14:51:13,480 - httpx - INFO


Strategy: RAG + Reranking

Query: Find dark psychological thrillers

Time: 19.13s

Answer: Based on the provided information, "The Babadook" is a notable dark psychological thriller. It explores themes of grief and tragedy through the lens of horror, featuring an iconic boogeyman and a chilling performance by Essie Davis. Additionally, "Sweet Virginia" is mentioned as a suspense-filled thriller that delves into themes of loneliness and despair, making it another option in the dark psychological thriller category.

Top 5 Retrieved:

  The Babadook - 2014(initial score=-1.0112135410308838)

 content: Review: This psychological thriller from gifted first-time filmmaker Jennifer Kent will have you climbing the walls simply by plumbing the violence of...

  rerank score: 0.900

  The Babadook - 2014(initial score=-0.9476317167282104)

 content: Review: This Australian thriller is refreshing because it smartly generates its chills from true psychological horrors rather than jump scares or t

#### RAG + HyDE

In [19]:
print("="*60)
print("DEMO 3: RAG + HyDE")
print("="*60)

# Build chain with HyDE
hyde_chain = MovieRAGChain(
    plots_path=PLOTS_PATH,
    reviews_path=REVIEWS_PATH,
    max_movies=MAX_MOVIES,
    use_custom_retriever=True,
    use_custom_chunk=True,
    custom_prompt=ZERO_SHOT_QA_PROMPT,
    k=5,
    use_hyde=True,
    hyde_model="gpt-4o-mini",    
)

hyde_chain.build()

# Test HyDE on ambiguous query
ambiguous_query = "Movies like that dream one"

result = hyde_chain.query(ambiguous_query)

run(hyde_chain, "Movies like that dream one", "RAG + HyDE")

DEMO 3: RAG + HyDE
✓ MovieRAGChain initialized
  Retriever type: custom + HyDE
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...


2025-10-14 15:05:36,457 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: mps
2025-10-14 15:05:36,459 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2. Chunking with custom func...

Chunking documents...
Chunked 883 docs → 10114 chunks using 'sentence' strategy.

Building base retriever...

3. Building custom retriever...
Loading embedding model: text-embedding-3-small (provider: openai)
✓ Model loaded (dimension: 1536)
✓ FaissDenseRetriever initialized (index_type=flat)
Generating embeddings for 10114 documents...


2025-10-14 15:05:44,099 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:05:48,705 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:05:53,520 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:05:57,642 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:06:03,357 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:06:05,222 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-14 15:06:05,226 - openai._base_client - INFO - Retrying request to /embeddings in 9.239000 seconds
2025-10-14 15:06:19,057 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:06:21,080 - httpx - INFO - HTTP Request: POST https://api.o

Embeddings generated
Saving index...
✓ Added 10114 documents to FAISS index
  Index size: 10114

4. Adding HyDE query transformation...
✓ HyDERetriever initialized (model=gpt-4o-mini)

5. Creating QA chain...

✓ RAG Pipeline Built!


2025-10-14 15:06:37,448 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:06:37,858 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:06:40,009 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:06:45,128 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:06:45,743 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:06:47,790 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Strategy: RAG + HyDE

Query: Movies like that dream one

Time: 7.78s

Answer: Movies like "Anomalisa" include "Being John Malkovich" and "Eternal Sunshine of the Spotless Mind," both of which feature the acerbic brilliance of Charlie Kaufman. Additionally, "Anomalisa" is noted for its dreamlike quality and emotional depth, similar to other creatively ambitious films that explore personal and social maturation. If you're looking for something that combines surreal imagery with profound themes, these films would be a good fit.

Top 5 Retrieved:

  hyde query: When discussing movies akin to the ethereal qualities of "that dream one," we often gravitate towards films that weave intricate narratives with surreal elements, blurring the lines between reality and imagination. A prime example is "Inception," directed by Christopher Nolan, which explores the depths of the subconscious through a heist-like structure that delves into dreams within dreams. Similarly, "Eternal Sunshine of the Spotl

In [23]:
print("="*60)
print("same query, no hyde")
print("="*60)

# Build chain with HyDE
hyde_chain = MovieRAGChain(
    plots_path=PLOTS_PATH,
    reviews_path=REVIEWS_PATH,
    max_movies=MAX_MOVIES,
    use_custom_retriever=True,
    use_custom_chunk=True,
    custom_prompt=ZERO_SHOT_QA_PROMPT,
    k=5,   
)

hyde_chain.build()

# Test HyDE on ambiguous query
ambiguous_query = "Movies like that dream one"

result = hyde_chain.query(ambiguous_query)

run(hyde_chain, "Movies like that dream one", "RAG")

same query, no hyde
✓ MovieRAGChain initialized
  Retriever type: custom
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...


2025-10-14 15:09:03,985 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: mps
2025-10-14 15:09:03,986 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2. Chunking with custom func...

Chunking documents...
Chunked 883 docs → 10114 chunks using 'sentence' strategy.

Building base retriever...

3. Building custom retriever...
Loading embedding model: text-embedding-3-small (provider: openai)
✓ Model loaded (dimension: 1536)
✓ FaissDenseRetriever initialized (index_type=flat)
Generating embeddings for 10114 documents...


2025-10-14 15:09:10,221 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:09:14,776 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:09:19,403 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:09:23,170 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:09:24,870 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-14 15:09:24,874 - openai._base_client - INFO - Retrying request to /embeddings in 0.232000 seconds
2025-10-14 15:09:28,154 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:09:30,096 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-14 15:09:30,099 - openai._base_client - INFO - Ret

Embeddings generated
Saving index...
✓ Added 10114 documents to FAISS index
  Index size: 10114

5. Creating QA chain...

✓ RAG Pipeline Built!


2025-10-14 15:09:56,500 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:10:00,035 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:10:00,343 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:10:04,201 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Strategy: RAG

Query: Movies like that dream one

Time: 4.11s

Answer: If you're looking for movies similar to "Unconscious," which features dreamlike elements and explores themes of personal and social maturation, you might enjoy "Cirque Du Soleil: Worlds Away." This film also presents a dreamlike experience as it follows a young woman who enters a fantastical world after a carnival accident, filled with enchanting performances and surreal imagery. Both films share a creative and imaginative approach to storytelling.

Top 5 Retrieved:

  Howl's Moving Castle - 2005(initial score=-1.1036126613616943)

 content: Review: The director's grasp of animation techniques is as impressive as the depth and substance of his imaginary worlds.
Review: One of the summer's tastiest treats...a beautifully wrought, intoxicatingly dreamlike parable of personal and social maturation.
Review: The weirdest dream or acid trip you'll ever experience, broadcasted on the big screen by a team of creative crani

#### RAG + HyDE + Reranking

In [24]:
print("="*60)
print("DEMO 4: All Features (HyDE + Reranking)")
print("="*60)

# Build chain with all features
full_chain = MovieRAGChain(
    plots_path=PLOTS_PATH,
    reviews_path=REVIEWS_PATH,
    max_movies=MAX_MOVIES,
    use_custom_retriever=True,
    use_custom_chunk=True,
    custom_prompt=ZERO_SHOT_QA_PROMPT,
    k=5,
    use_hyde=True,
    hyde_model="gpt-4o-mini",
    use_reranking=True,
    reranker_cfg={'type':'llm'},
    initial_k=20, 
)

full_chain.build()

run(full_chain, "recommend mind-bending sci-fi with great visuals", "RAG + HyDE + Reranking")

DEMO 4: All Features (HyDE + Reranking)
✓ MovieRAGChain initialized
  Retriever type: custom + reranking + HyDE
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...


2025-10-14 15:10:53,554 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: mps
2025-10-14 15:10:53,555 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2. Chunking with custom func...

Chunking documents...
Chunked 883 docs → 10114 chunks using 'sentence' strategy.

Building base retriever...

3. Building custom retriever...
Loading embedding model: text-embedding-3-small (provider: openai)
✓ Model loaded (dimension: 1536)
✓ FaissDenseRetriever initialized (index_type=flat)
Generating embeddings for 10114 documents...


2025-10-14 15:10:59,596 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:11:04,713 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:12:40,982 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:12:45,864 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:12:51,006 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:12:56,024 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:12:59,503 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Embeddings generated
Saving index...
✓ Added 10114 documents to FAISS index
  Index size: 10114

4. Adding HyDE query transformation...
✓ HyDERetriever initialized (model=gpt-4o-mini)

5. Adding reranking: 20 → 5 docs...
✓ LLMReranker initialized: gpt-4o-mini
✓ Created LLM reranking retriever (top_k=5)

5. Creating QA chain...

✓ RAG Pipeline Built!


2025-10-14 15:13:05,238 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:13:05,656 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:13:07,390 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:13:07,900 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:13:08,412 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:13:10,666 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:13:11,119 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:13:13,226 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Strategy: RAG + HyDE + Reranking

Query: recommend mind-bending sci-fi with great visuals

Time: 12.09s

Answer: For mind-bending sci-fi with great visuals, I recommend "Alien." Directed by Ridley Scott, it is celebrated for its stunning visuals and innovative design work by H.R. Giger. The film creates a haunting atmosphere and features a unique blend of horror and science fiction that remains impactful. While it primarily focuses on suspense and terror, its visual storytelling and original settings make it a remarkable cinematic experience.

Top 5 Retrieved:

  hyde query: For a truly mind-bending sci-fi experience that dazzles with exceptional visuals, I highly recommend "Interstellar" directed by Christopher Nolan. This cinematic masterpiece transcends conventional storytelling through its intricate exploration of time, space, and the human condition. The film's stunning visual effects, particularly the awe-inspiring depiction of black holes and the surreal landscapes of distant p

#### Comparison of Strategies

In [27]:
print("="*60)
print("COMPARISON: RAG Strategies")
print("="*60)

test_query = "intellectual movies with complex plots that defy logic"
print(f"\nTest Query: {test_query}\n")

# Define strategies
strategies = [
    {"name": "Baseline", "config": {}},
    {"name": "With Reranking", "config": {"use_reranking": True, "reranker_cfg": {'type':'llm'}, "initial_k": 20}},
    {"name": "With HyDE", "config": {"use_hyde": True, "hyde_model": "gpt-4o-mini"}},
    {"name": "HyDE + Reranking", "config": {"use_hyde": True, "hyde_model": "gpt-4o-mini", "use_reranking": True, "reranker_cfg": {'type':'llm'}, "initial_k": 20}}
]

results = []

# Run comparison
for strategy in strategies:
    print(f"\nTesting: {strategy['name']}...")

    # Build chain
    chain = MovieRAGChain(
        plots_path=PLOTS_PATH,
        reviews_path=REVIEWS_PATH,
        max_movies=MAX_MOVIES,
        use_custom_retriever=True,
        use_custom_chunk=True,
        custom_prompt=ZERO_SHOT_QA_PROMPT,
        k=5,
        **strategy['config']
    )
    chain.build()

    run(chain, test_query, strategy['name'])

COMPARISON: RAG Strategies

Test Query: intellectual movies with complex plots that defy logic


Testing: Baseline...
✓ MovieRAGChain initialized
  Retriever type: custom
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...


2025-10-14 15:18:02,722 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: mps
2025-10-14 15:18:02,723 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2. Chunking with custom func...

Chunking documents...
Chunked 883 docs → 10114 chunks using 'sentence' strategy.

Building base retriever...

3. Building custom retriever...
Loading embedding model: text-embedding-3-small (provider: openai)
✓ Model loaded (dimension: 1536)
✓ FaissDenseRetriever initialized (index_type=flat)
Generating embeddings for 10114 documents...


2025-10-14 15:18:08,872 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:18:13,796 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:18:18,277 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:18:23,192 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:18:26,674 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:18:28,515 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-14 15:18:28,518 - openai._base_client - INFO - Retrying request to /embeddings in 10.781000 seconds
2025-10-14 15:18:42,240 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:18:44,183 - httpx - INFO - HTTP Request: POST https://api.

Embeddings generated
Saving index...
✓ Added 10114 documents to FAISS index
  Index size: 10114

5. Creating QA chain...

✓ RAG Pipeline Built!


...

Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2. Chunking with custom func...

Chunking documents...
Chunked 883 docs → 10114 chunks using 'sentence' strategy.

Building base retriever...

3. Building custom retriever...
Loading embedding model: text-embedding-3-small (provider: openai)
✓ Model loaded (dimension: 1536)
✓ FaissDenseRetriever initialized (index_type=flat)
Generating embeddings for 10114 documents...


2025-10-14 15:22:53,426 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:22:57,412 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:22:58,781 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-14 15:22:58,784 - openai._base_client - INFO - Retrying request to /embeddings in 7.679000 seconds
2025-10-14 15:23:09,300 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:23:12,064 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-14 15:23:12,067 - openai._base_client - INFO - Retrying request to /embeddings in 9.634000 seconds
2025-10-14 15:23:26,093 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:23:27,464 - httpx - INFO - HTTP Request: POST https:

Embeddings generated
Saving index...
✓ Added 10114 documents to FAISS index
  Index size: 10114

4. Adding HyDE query transformation...
✓ HyDERetriever initialized (model=gpt-4o-mini)

5. Adding reranking: 20 → 5 docs...
✓ LLMReranker initialized: gpt-4o-mini
✓ Created LLM reranking retriever (top_k=5)

5. Creating QA chain...

✓ RAG Pipeline Built!


2025-10-14 15:24:11,868 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:24:12,411 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 15:24:13,985 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:24:14,526 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:24:15,854 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:24:16,681 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:24:17,433 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-14 15:24:20,539 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Strategy: HyDE + Reranking

Query: intellectual movies with complex plots that defy logic

Time: 13.09s

Answer: For intellectual movies with complex plots that defy logic, consider the following:

1. **eXistenZ** - Written by David Cronenberg, this film creates a bizarre world where organic and inorganic elements blend, keeping viewers guessing throughout its mind-bending narrative.

2. **Anomalisa** - Directed by Charlie Kaufman, this film presents a deeply human story that explores existential themes, requiring multiple viewings to fully digest its complexities.

3. **Being John Malkovich** and **Eternal Sunshine of the Spotless Mind** - Both films, scripted by Charlie Kaufman, showcase acerbic brilliance and innovative storytelling that challenge conventional narratives.

These films are known for their intricate plots and thought-provoking themes, making them suitable for viewers seeking intellectually stimulating cinema.

Top 5 Retrieved:

  hyde query: Intellectual movies with 

#### Streaming Responses

In [6]:
print("="*60)
print("DEMO 5: Streaming Responses")
print("="*60)

# Build base chain
base = MovieRAGChain(
    plots_path=PLOTS_PATH,
    reviews_path=REVIEWS_PATH,
    max_movies=MAX_MOVIES,
    use_custom_retriever=True,
    use_custom_chunk=True,
    custom_prompt=ZERO_SHOT_QA_PROMPT,
    k=5,
    use_reranking=True
)
base.build()

# Test real streaming
print("="*60)
print("REAL STREAMING (with inline citations)")
print("="*60)

DEMO 5: Streaming Responses
✓ MovieRAGChain initialized
  Retriever type: custom + reranking
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...
Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2. Chunking with custom func...

Chunking documents...


2025-10-14 18:11:34,574 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: mps
2025-10-14 18:11:34,575 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Chunked 883 docs → 10114 chunks using 'sentence' strategy.

Building base retriever...

3. Building custom retriever...
Loading embedding model: text-embedding-3-small (provider: openai)
✓ Model loaded (dimension: 1536)
✓ FaissDenseRetriever initialized (index_type=flat)
Generating embeddings for 10114 documents...


2025-10-14 18:11:41,656 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 18:11:46,513 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 18:11:50,414 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 18:11:55,727 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 18:11:59,732 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 18:12:01,982 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
2025-10-14 18:12:01,993 - openai._base_client - INFO - Retrying request to /embeddings in 8.863000 seconds
2025-10-14 18:12:14,066 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-14 18:12:15,951 - httpx - INFO - HTTP Request: POST https://api.o

Embeddings generated
Saving index...
✓ Added 10114 documents to FAISS index
  Index size: 10114

4. Adding reranking: 20 → 5 docs...


2025-10-14 18:12:28,062 - sentence_transformers.cross_encoder.CrossEncoder - INFO - Use pytorch device: mps


✓ CrossEncoderReranker loaded: cross-encoder/ms-marco-MiniLM-L-6-v2
✓ Created cross-encoder reranking retriever (top_k=5)

5. Creating QA chain...

✓ RAG Pipeline Built!
REAL STREAMING (with inline citations)


In [18]:
streaming = CitationStreamingWrapper(base)
_ = streaming.stream("What makes Christopher Nolan's films special?")

Question: What makes Christopher Nolan's films special?

Answer: 

2025-10-14 17:49:55,849 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-10-14 17:49:56,992 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Christopher Nolan's films are often characterized by their intricate storytelling, emotional depth, and innovative visual style. While the provided information does not directly discuss Nolan's films, it does mention that "Transcendence," directed by Wally Pfister, is compared to a Nolan film in terms of visual appeal but lacks the substance and emotional resonance typically found in Nolan's work [4]. Nolan is known for his ability to blend complex narratives with profound themes, which sets his films apart from others in the genre.

Citations:

[4] Transcendence, Source: review, Preview: Review: [Transcendence] is more than just a big budget version of Siri Goes Wild.
Review: This would all fit better in a Wired op-ed than in a movie.
Review: Transcendence becomes weirdly successful as a sort of cockeyed analogy for the gospel and the painful, tragic, violent beauty of the Incarnation and Christ's crucifixion.
Review: "Transcendence" gets an "A" for effort, competently melding art-fil

In [19]:
streaming = CitationStreamingWrapper(base)
_ = streaming.stream("Can you guide me to spiritual cinema?")

Question: Can you guide me to spiritual cinema?

Answer: 

2025-10-14 17:50:54,543 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-10-14 17:50:56,217 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Certainly! Here are some notable films that explore spiritual themes:

1. **Higher Ground** - Directed by Vera Farmiga, this film presents a coming-of-faith tale that delves into the complexities of doubt and faith. It challenges viewers to reflect on their own spiritual journeys and offers a nuanced portrayal of a fundamental Christian grappling with her beliefs [1][3].

2. **Ushpizin** - This film celebrates love and the possibility of second chances within a cloistered society. It balances humor and spiritual wisdom, drawing audiences into its unique cultural context while addressing universal themes of faith, family, and friendship [4].

3. **A First-Person Account of Belief** - This film provides a dignified exploration of how belief systems are created and evolve. It offers a respectful and nuanced portrayal of religion, inviting viewers to engage deeply with its themes [5].

These films each provide a different perspective on spirituality and faith, making them excellent choices

#### LangSmith Tracing

In [ ]:
print("="*60)
print("DEMO 6: LangSmith Tracing")
print("="*60)

from langsmith import tracing_context

setup_langsmith('test-project')

# Build chain
trace_chain = base

# Run traced queries
test_queries = [
    "sci-fi with time travel",
    "highly rated comedies",
    "movies about dreams"
]

for i, query in enumerate(test_queries, 1):
    with tracing_context(project_name="test"):
        result = trace_chain.query(query)
        print(f"\nQuery {i} traced: {query}")
        print(f"   Answer: {result['answer']}")

print("\nView traces at: https://smith.langchain.com/")